In [10]:
# coding: utf-8


import pandas as pd
import numpy as np
import re
import tkinter
from tkinter import filedialog


###
###
### IMPORT RAW DATA
###
###



# import the excel file
rh = r"C:\Users\mkreidler\Desktop\35w\rhino.xlsx"

# convert files to dataframes
rhino = pd.read_excel(rh)

df = rhino

# clean data
df.x = pd.to_numeric(df.x)
df.y = pd.to_numeric(df.y)
df.z = pd.to_numeric(df.z)

# creating the framework
df["new"] = ""
df["co"] = ""
df["priority_ctrac"] = ""
df["swing_drop"] = ""
df["faulty"] = ""
df["remediation"] = ""
df["deduct"] = ""
df["floor"] = ""
df["elevation"] = ""
df["project"] = ""
rhino["bv_survey"] = ""
rhino["co11_12"] = ""
rhino["mk_priority"] = ""
rhino["ct_val"] = 1
rhino["ct_order"] = ""

# creating definitions
df.new = df.layer.apply(lambda x: "To Produce" if "NEW" in str(x) else "Made")   #this defines and executes at once


def find_co(x):
    pat = r"CO[\s]*([0-9]+)"
    result = re.search(pat, str(x))
    if result:
        return int(result.group(1))


def find_priority(x):
    pat = r"P([0-9]+)"
    result = re.search(pat, str(x))
    if result:
        return int(result.group(1))


def get_drop(x):
    pat = r"drop\s*([a-z]*\d*)"
    match = re.search(pat, str(x), flags=re.IGNORECASE)

    if match:
        # print("Found Match")
        return match.group(1)
    else:
        # print("Not Found")
        return None


def find_faulty(x):
    pat = r"faulty"
    result = re.search(pat, str(x), flags=re.IGNORECASE)
    if result:
        return "faulty"


def find_remediation(x):
    pat = r"remediation"
    result = re.search(pat, str(x), flags=re.IGNORECASE)
    if result:
        return "re"


def find_deduct(x):
    pat = r"deduct"
    result = re.search(pat, str(x), flags=re.IGNORECASE)
    if result:
        return "deduct"


def find_floor(x):
    pat = r"floor\s*(\d*)"
    result = re.search(pat, str(x), flags=re.IGNORECASE)
    if result:
        return int(result.group(1))


def find_elevation(x):
    pat = r"([a-zA-Z]*)\s*ELEVATION"
    result = re.search(pat, str(x), flags=re.IGNORECASE)
    if result:
        return result.group(1)


def find_project(x):
    pat = r"'project\s+(.*)'"
    result = re.search(pat, str(x), flags=re.IGNORECASE)
    if result:
        return result.group(1)
    else:
        print("not in project")


def check_survey(x):
    if x not in ["S2", "S3", "S4"]:
        return "bv_surveyed"
    else:
        return "survey required"


def find_co11_12(x):
    if x in [11, 12]:
        return "NIS"
    else:
        return 


# executing the functions to parse/assign data
df.co = df.layer.apply(find_co)
df.priority_ctrac = df.layer.apply(find_priority)
df.swing_drop = df.zones.apply(get_drop)
df.faulty = df.layer.apply(find_faulty)
df.remediation = df.layer.apply(find_remediation)
df.deduct = df.layer.apply(find_deduct)
df.floor = df.zones.apply(find_floor)
df.elevation = df.zones.apply(find_elevation)
df.project = df.zones.apply(find_project)
df.bv_survey = df.swing_drop.apply(check_survey)
df.co11_12 = df.co.apply(find_co11_12)

# assign the mk method of sorting drops
df.loc[df.swing_drop == "E1", "mk_priority"] = 1
df.loc[df.swing_drop == "S1", "mk_priority"] = 2
df.loc[df.swing_drop == "S5", "mk_priority"] = 3
df.loc[df.swing_drop == "W1", "mk_priority"] = 4
df.loc[df.swing_drop == "W2", "mk_priority"] = 5
df.loc[df.swing_drop == "W3", "mk_priority"] = 6
df.loc[df.swing_drop == "S4", "mk_priority"] = 7
df.loc[df.swing_drop == "S3", "mk_priority"] = 8
df.loc[df.swing_drop == "S2", "mk_priority"] = 9

# sort values and hold that order with cumsum()
df = df.sort_values(["bv_survey", "co11_12", "mk_priority", "z", "new"], ascending=[True, False, True, True, True])
df["ct_order"] = df.ct_val.cumsum()

#df["shipped"] = df.priority_ctrac.apply(lambda x: "re" if str(x).upper() == "SHIPPED" else None) ###does not seem to be any shipped units in rhino layer

rh = df[["name", "guid", "co11_12", "faulty", "remediation"]].groupby("name").count().reset_index()

#####SAVE



#####
##### PARSE CTRAC
#####


#import excel
ct = r"C:\Users\mkreidler\Desktop\35w\ctrac.xlsx"
ct_old = r"C:\Users\mkreidler\Desktop\35w\ctrac_old.xlsx"

#convert to df
ctrac = pd.read_excel(ct)
ctrac_old = pd.read_excel(ct_old)

#groupby names
ctrac_new_join = ctrac.groupby("name").sum().reset_index()
ctrac_old_join = ctrac_old.groupby("name").sum().reset_index()

#merge ctracs
both = ctrac_new_join.merge(ctrac_old_join, on="name", how="outer").fillna(0)
both = both.rename(index=str, columns={"quantity_ctrac": "ct_qty_P19-9999", "quantity_old":"ct_qty_P14-0436"})


###
###
### MERGE BOTH
###
###

full = rh.merge(both, on="name", how="outer").fillna(0)
full.to_excel(r"C:\Users\mkreidler\Desktop\35w\full_name_study2.xlsx")

In [7]:
#import excel
ct = r"C:\Users\mkreidler\Desktop\35w\ctrac.xlsx"
ct_old = r"C:\Users\mkreidler\Desktop\35w\ctrac_old.xlsx"

#convert to df
ctrac = pd.read_excel(ct)
ctrac_old = pd.read_excel(ct_old)

#groupby names
ctrac_new_join = ctrac.groupby("name").sum().reset_index()
ctrac_old_join = ctrac_old.groupby("name").sum().reset_index()

#merge ctracs
both = ctrac_new_join.merge(ctrac_old_join, on="name", how="outer").fillna(0)
both = both.rename(index=str, columns={"quantity_ctrac": "ct_qty_P19-9999", "quantity_old":"ct_qty_P14-0436"})

both.head()

,name,ct_qty_P19-9999,ct_qty_P14-0436
0,2B1,4.0,2.0
1,2B10,2.0,1.0
2,2B11,1.0,2.0
3,2B12,1.0,1.0
4,2B13,1.0,1.0


In [9]:
ctrac_old

,name,quantity_old,priority_old
0,2B1,0,VIF
1,2B1,2,1
2,2B10,1,1
3,2B11,1,CO5
4,2B11,1,1
5,2B12,1,1
6,2B13,1,1
7,2B1A,1,1
8,2B2,1,1
9,2B3A,2,1
